In [1]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np
import pandas as pd

coor_data = pd.read_csv('Data/coordata.csv')
date_data = pd.read_csv('Data/datedata.csv')
meta_data = pd.read_csv('Data/metadata.csv')
bus_stop_data = pd.read_csv('Data/bus_stop.csv')

# general = pd.read_csv('Data/ScooterData_July15_Sept27_2019.csv')

data_set = coor_data.join(bus_stop_data.join(date_data.join(meta_data)))


Y_data = data_set[['endx', 'endy']].to_numpy()
X_data = data_set.drop(columns=['endx', 'endy','start_date', 'end_near_stop']).to_numpy()

I:\Users\PC\anaconda3\envs\my_env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
I:\Users\PC\anaconda3\envs\my_env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
I:\Users\PC\anaconda3\envs\my_env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
I:\Users\PC\anaconda3\envs\my_env\lib\site-packages\tensorflow\python\framework

In [29]:
import progressbar

X_sets_by_day = []
Y_sets_by_day = []

hr = 0
x = []
y = []

compare_hr_before = 0
compare_hr_current = 0
for i in progressbar.progressbar(range(len(X_data))):
    compare_hr_current = data_set['start_hour'][i]
    if (compare_hr_before == 23 and compare_hr_current == 0):
        hr = 0
        X_sets_by_day.append(x)
        Y_sets_by_day.append(y)
        x = []
        y = []
    compare_hr_before = data_set['start_hour'][i]
    x.append(X_data[i])
    y.append(Y_data[i])
    if i == (len(X_data) - 1):
        X_sets_by_day.append(x)
        Y_sets_by_day.append(y)

100% (459478 of 459478) |################| Elapsed Time: 0:00:07 Time:  0:00:07


In [80]:
X_train_set = []
Y_train_set = []
for i in progressbar.progressbar(range(1)):
    for j in range(len(X_sets_by_day[i])):
        X_train_set.append(X_sets_by_day[i][j])
        Y_train_set.append(Y_sets_by_day[i][j])

X_test_set = []
Y_test_set = []
for i in progressbar.progressbar(range(1,len(X_sets_by_day))):
    for j in range(len(X_sets_by_day[i])):
        X_test_set.append(X_sets_by_day[i][j])
        Y_test_set.append(Y_sets_by_day[i][j])
        
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.callbacks import EarlyStopping

scx = MinMaxScaler(feature_range=(0,1))
X_train_set = scx.fit_transform(X_train_set)
X_test_set = scx.transform(X_test_set)

scy = MinMaxScaler(feature_range=(0,1))
Y_train_set = scy.fit_transform(Y_train_set)
Y_test_set = scy.transform(Y_test_set)

100% (1 of 1) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (74 of 74) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [ ]:
from tensorflow.keras import layers, Model, regularizers
from tensorflow.python.keras.layers.core import Dense, Activation, Dropout
from tensorflow.python.keras.layers.recurrent import LSTM
inputA = layers.Input(shape=(4))
inputB = layers.Input(shape=(60))

# x = Dense(50, activation="relu")(inputA)
x = Dense(15)(inputA)
x = Dense(30)(x)
x = Model(inputs=inputA, outputs=x)
y = Dense(62)(inputB)
y = Dense(70)(y)
y = Model(inputs=inputB, outputs=y)

combined = layers.concatenate([x.output, y.output])

final = Dense(70, activation="softmax")(combined)
# b = Dense(2)(final)
# combined = layers.concatenate([final, b])
a = Dense(2)(final)

model = Model(inputs=[x.input,y.input], outputs=a)
model.summary()



model.compile(loss="mean_absolute_error", optimizer="adam")

x_test = X_test_set
y_test = Y_test_set

x_train = X_train_set
y_train = Y_train_set


history = model.fit([x_train[:,0:4],x_train[:,4:]], y_train, epochs=50, verbose=2)
                
for i in range(len(X_sets_by_day)):
    tr_xd_tr = scx.transform(np.array(X_sets_by_day[i]))
    X_train_set = np.append(X_train_set, tr_xd_tr, axis = 0)
    tr_yd_tr = scy.transform(np.array(Y_sets_by_day[i]))
    Y_train_set = np.append(Y_train_set, tr_yd_tr, axis = 0)
    
    x_train = X_train_set
    y_train = Y_train_set
    history = model.fit([x_train[:,0:4],x_train[:,4:]], y_train, epochs=50, verbose=2)
    print('current at: '+str(i))
    
    if len(X_train_set) > 100000:
        X_train_set = X_train_set[len(X_train_set)-100000:len(X_train_set)-100000]
        Y_train_set = Y_train_set[len(X_train_set)-100000:len(Y_train_set)-100000]
#     for j in range(len(X_sets_by_day[i])):
#         X_train_new_set.append(X_sets_by_day[i][j])
#         Y_train_new_set.append(Y_sets_by_day[i][j])

Model: "model_62"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_41 (InputLayer)           [(None, 4)]          0                                            
__________________________________________________________________________________________________
input_42 (InputLayer)           [(None, 60)]         0                                            
__________________________________________________________________________________________________
dense_120 (Dense)               (None, 15)           75          input_41[0][0]                   
__________________________________________________________________________________________________
dense_122 (Dense)               (None, 62)           3782        input_42[0][0]                   
___________________________________________________________________________________________

In [ ]:
def haversine(coord1: object, coord2: object):
    import math

    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    meters = round(meters, 5)
    km = round(km, 5)
    return km

In [ ]:

    x_test = sc.fit_transform(np.array(X_test_set))
    y_test = sc.fit_transform(np.array(Y_test_set))
    
    result_Y = model.predict([x_test[:,0:4],x_test[:,4:]])
    result_Y_all = sc.inverse_transform(result_Y)
    y_test_all = sc.inverse_transform(y_test)
    error_all = 0
    for i in (range(len(y_test_all))):
        error_all = error_all + haversine(result_Y_all[i],y_test_all[i])
        error_avg = error_all/len(y_test_all)
    print(round(error_avg, 3))
